In [ ]:
!pip install pydantic[email]

In [ ]:
pip install mcp_odoo/.

In [7]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [1]:
from src.data.data_retriever import DataRetriever
from src.data.odoo_connector import OdooConnection
import asyncio
from config.settings import settings
import nest_asyncio
import pandas as pd
import numpy as np
nest_asyncio.apply()

In [2]:
odoo_connection = OdooConnection()
asyncio.run(odoo_connection.connect())
data_retriever = DataRetriever(odoo_connection=odoo_connection)

Connected to Odoo as albert.gil@yourtechtribe.com (uid: 430)
Odoo server version: {'server_version': '16.0+e-20250313', 'server_version_info': [16, 0, 0, 'final', 0, 'e'], 'server_serie': '16.0', 'protocol_version': 1}


# Datos empresas

In [15]:
asyncio.run(data_retriever.get_all_companies())

[Company(id=1, name='Grupo Viko Digital Marketing, S.A.', currency_id=(1, 'EUR')),
 Company(id=3, name='Elogia Media S.L.', currency_id=(1, 'EUR')),
 Company(id=2, name='Ibrands Medios Interactivos SL', currency_id=(1, 'EUR')),
 Company(id=5, name='Kraz Data Solutions SL', currency_id=(1, 'EUR')),
 Company(id=6, name='Marketing4ecommerce Digital Content SL', currency_id=(1, 'EUR')),
 Company(id=13, name='Octoplus Digital Shelf Optimization SL', currency_id=(1, 'EUR')),
 Company(id=7, name='Tandem Trade Marketing SL', currency_id=(1, 'EUR')),
 Company(id=11, name='DigitalPla2021, S.L.', currency_id=(1, 'EUR')),
 Company(id=14, name='Ideas y Estrategia Digital SL', currency_id=(1, 'EUR')),
 Company(id=8, name='INICIATIVAS VIRTUALES DE MEXICO', currency_id=(33, 'MXN')),
 Company(id=12, name='IBRANDS MEDIOS INTERACTIVOS DE MEXICO', currency_id=(33, 'MXN')),
 Company(id=9, name='MITTUM MARKETING RELACIONAL', currency_id=(33, 'MXN'))]

# Datos facturas

In [32]:
company_id = 14
invoices = asyncio.run(data_retriever.get_all_outbound_invoices(company_id))
invoices_df = pd.DataFrame([i.to_dict() for i in invoices])

Recuperadas 500 facturas, total: 500
Recuperadas 95 facturas, total: 595


In [33]:
invoices_df.describe()

,id,amount_total,amount_residual,days_overdue
count,595.000000,595.000000,595.000000,595.000000
mean,143420.842017,5338.248672,912.600571,7.858824
std,21899.350306,8546.715734,4151.656529,13.038414
min,121169.000000,108.730000,0.000000,-1.000000
25%,130328.500000,955.900000,0.000000,0.000000
50%,130477.000000,2178.000000,0.000000,4.000000
75%,157948.500000,6198.225000,0.000000,11.500000
max,194734.000000,90096.600000,41964.870000,103.000000


In [34]:
invoices_df.columns.values

array(['id', 'name', 'move_type', 'payment_state', 'company_id',
       'partner_id', 'currency_id', 'amount_total', 'amount_residual',
       'invoice_date', 'invoice_date_due', 'journal_id', 'payment_dates',
       'paid_late', 'days_overdue'], dtype=object)

In [35]:
invoices_df["payment_dates"].dtypes

dtype('O')

In [36]:
invoices_df["payment_state"].value_counts()

payment_state
paid        515
not_paid     80
Name: count, dtype: int64

In [37]:
invoices_df["paid_late"].value_counts()

paid_late
True     365
False    150
Name: count, dtype: int64

In [38]:
invoices_df["partner_id"].value_counts()

partner_id
(13557, Chiesi España, S.A)                          124
(13548, Esteve Pharmaceuticals SA)                   113
(13558, Elanco Spain S.L.)                           106
(8933, Zambon, S.A.U.)                                66
(13559, Kern Pharma S.L.)                             47
(13549, Swedish Orphan Biovitrum S.L.)                34
(13542, Laboratorio Reig Jofre, S.A.)                 21
(13547, Alexion Pharma Nordics AB)                    18
(13543, Alexion Pharma GMBH)                          11
(14595, Swedish Orphan Biovitrum AB (publ))            8
(7583, Reckitt Benckiser Healthcare, S.A.)             8
(14476, Faes Farma S.A.)                               7
(13654, Atika Pharma S.L.)                             7
(13545, Elanco AH Portugal Unipessoal Lda.)            6
(13544, Angelini Pharma Portugal, Unipessoal Lda)      3
(13553, AdSalutem Lullaai S.L)                         2
(14627, Asociación Española Contra el Cáncer)          2
(10, Elogia Media S.

In [16]:
invoices_df["currency_id"].value_counts()

currency_id
(1, EUR)      12388
(18, SEK)         9
(142, GBP)        1
Name: count, dtype: int64

In [10]:
invoices_df

,id,name,move_type,payment_state,company_id,partner_id,currency_id,amount_total,amount_residual,invoice_date,invoice_date_due,journal_id,payment_dates,paid_late,days_overdue
0,196743,FVE/2025/00376,out_invoice,not_paid,"(3, Elogia Media S.L.)","(8920, Boehringer Ingelheim España, S.A.)","(1, EUR)",242.00,242.00,2025-06-17,2025-08-16,"(28, Facturas ventas Elogia)",None,None,-1
1,196654,FVE/2025/00375,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",453.75,453.75,2025-06-12,2025-08-11,"(28, Facturas ventas Elogia)",None,None,-1
2,196649,FVE/2025/00374,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",907.50,907.50,2025-06-12,2025-08-11,"(28, Facturas ventas Elogia)",None,None,-1
3,196631,FVE/2025/00373,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",1210.00,1210.00,2025-06-11,2025-08-10,"(28, Facturas ventas Elogia)",None,None,-1
4,196628,FVE/2025/00372,out_invoice,not_paid,"(3, Elogia Media S.L.)","(14945, Olistic Research Labs, S.L.)","(1, EUR)",1210.00,1210.00,2025-06-11,2025-08-10,"(28, Facturas ventas Elogia)",None,None,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,9691,INV1/2022/00005,out_invoice,paid,"(3, Elogia Media S.L.)","(9247, Apoteca Natura Spa)","(1, EUR)",2250.00,0.00,2022-12-31,2023-01-30,"(340, Carga facturas venta Elogia)",2023-02-01,True,2
6344,9690,INV1/2022/00004,out_invoice,paid,"(3, Elogia Media S.L.)","(8301, Goiko Grill Group SL.)","(1, EUR)",11555.50,0.00,2022-12-31,2023-01-29,"(340, Carga facturas venta Elogia)",2023-02-17,True,19
6345,9689,INV1/2022/00003,out_invoice,paid,"(3, Elogia Media S.L.)","(7439, Boehringer Ingelheim Animal Health Espa...","(1, EUR)",1597.20,0.00,2022-12-31,2023-02-05,"(340, Carga facturas venta Elogia)",2023-03-01,True,24
6346,9688,INV1/2022/00002,out_invoice,paid,"(3, Elogia Media S.L.)","(8663, Irmaos Vila Nova SA)","(1, EUR)",12195.41,0.00,2022-12-31,2023-02-18,"(340, Carga facturas venta Elogia)",2023-02-21,True,3
